In [1]:
import os
import glob
import pandas as pd
import datetime
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
seltab_truth = '/home/ys587/__Data/__whistle/__whistle_dclde2020/metadata/DCLDE2020_DetectionData.xlsx'

In [4]:
# 'BD'=>18; 'STR'=>
species_id_dclde2020 = [2, 13, 15, 18, 21, 26, 31, 32, 33, 36]
species_dclde2020 = {18:1, 31:2, 13:4, 2:5, 36:7, 21:8, 15:9, 33:11, 26:16, 32:17}
score_thre = 0.2

species_dict = {'NO': 0, 'BD': 1, 'MH': 2, 'CD': 3, 'STR': 4, 'SPT': 5, 'SPIN': 6, 'PLT': 7, 'RD': 8, 'RT': 9,
                'WSD': 10, 'FKW': 11, 'BEL': 12, 'KW': 13, 'WBD': 14, 'DUSK': 15, 'FRA': 16, 'PKW': 17, 'LPLT': 18,
                'NAR': 19, 'CLY': 20, 'SPE': 21, 'ASP': 22}
species_list = list(species_dict.keys())

In [5]:
t1=list(species_dclde2020.values())
t1.sort()

In [6]:
t1

[1, 2, 4, 5, 7, 8, 9, 11, 16, 17]

In [7]:
species_lookup = pd.read_excel(seltab_truth, sheet_name='Species_Lookup')

In [8]:
detection_1705 =  pd.read_excel(seltab_truth, sheet_name='1705_OdontoceteDetections')
detection_1706 =  pd.read_excel(seltab_truth, sheet_name='1706_OdontoceteDetections')

In [9]:
detection_1705_species =  detection_1705[detection_1705['Species1ID'].isin(species_id_dclde2020)]
detection_1706_species =  detection_1706[detection_1706['Species1ID'].isin(species_id_dclde2020)]

In [10]:
detection_1705_species['Species1ID'].value_counts()
# no 18/BD, no 102/SPINNER

2     10
33     8
21     6
15     5
13     3
36     1
32     1
31     1
26     1
Name: Species1ID, dtype: int64

In [11]:
seltab_pred = '/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out'
seltab_list = glob.glob(os.path.join(seltab_pred, '*.txt'))

In [12]:
# seltab filename lists for 1705 & 1706, respectively
seltab_list_1705 = []
seltab_list_1706 = []
for ss in seltab_list:
    if '1705_' in ss:
        seltab_list_1705.append(ss)
    elif '1706' in ss:
        seltab_list_1706.append(ss)
seltab_list_1705.sort()
seltab_list_1706.sort()

In [14]:
detection_1705_species.iloc[:5]

,DetectionTimeStart_ShipLocal,DetectionTimeStart_UTC,DetectionTimeEnd_UTC,AcousticDetectionNumber,SightingNumber,Latitude,Longitude,FirstAngle,FirstDistance,BeamDistance,BeamTime_UTC,DetectionDistance,Species1ID,Species2ID,Species3ID,Var16,Var17,Var18
10,2017-08-31 17:52:23,2017-09-01 03:52:23,2017-09-01 03:58:05,77,34,19.490832,-156.079455,75.0,NaN,NaN,NaT,NaN,2,NaN,NaN,NaN,NaN,NaN
13,2017-09-01 17:21:40,2017-09-02 03:21:40,2017-09-02 04:43:25,82,35,18.543423,-159.123328,66.0,3800.0,3450.0,2017-09-02 03:42:15,3800.0,2,NaN,NaN,NaN,NaN,NaN
15,2017-09-02 07:28:17,2017-09-02 17:28:17,2017-09-02 19:32:13,84,38,19.489827,-159.729945,23.0,3150.0,960.0,NaT,3150.0,21,NaN,NaN,NaN,NaN,NaN
16,2017-09-02 09:24:09,2017-09-02 19:24:09,2017-09-02 20:24:44,85,39,19.511287,-159.618823,19.0,NaN,8700.0,NaT,NaN,13,NaN,NaN,NaN,NaN,NaN
17,2017-09-02 10:46:13,2017-09-02 20:46:13,2017-09-02 21:13:49,86,40,19.452268,-159.442957,71.0,NaN,NaN,NaT,NaN,32,NaN,NaN,NaN,NaN,NaN


# Deployment 1705

In [15]:
deploy='1705'

species_true_tot = []
species_pred = []
species_score = []
seltab_list_tot = []
# For each acoustic encounter
# for index, row in detection_1705_species_small.iterrows():
for index, row in detection_1705_species.iterrows():
    min_list0 = pd.DataFrame(columns=['NULL'], index=pd.date_range(row['DetectionTimeStart_UTC'], row['DetectionTimeEnd_UTC'],freq='1T')).index.strftime('%Y%m%d_%H%M').tolist()
    min_list = [deploy+'_'+mm for mm in min_list0]
    # ind_start = seltab_list_1705.index(min_list[0])
    print('\nFrom '+min_list[0]+' to '+min_list[-1])
    
    # find start and end indices
    for mm0 in range(len(seltab_list_1705)):
        mm = seltab_list_1705[mm0]
        if min_list[0] in mm:
            print('mm0: '+str(mm0))
            print(mm)
            for mm1 in range(mm0+1, mm0+120):
                mm_end = seltab_list_1705[mm1]
                if  min_list[-1] in mm_end:
                    print('mm1: '+str(mm1))
                    print(mm_end)
                    break
            break
    
    # truth species
    species_true_id = species_dclde2020[row['Species1ID']]
    print('True species id: '+str(species_true_id)+' '+species_list[species_true_id])
    species_true_encounter = np.ones(mm1-mm0+1)*species_true_id
    species_true_tot.append(species_true_encounter)  # <=
    
    
    # extract seltab filenames
    seltab_list = seltab_list_1705[mm0:mm1+1]
    for ss in seltab_list:
        print(ss)
        df_seltab = pd.read_csv(ss, delimiter='\t', header=0)
        species_score_curr = np.array(df_seltab.iloc[:, 13:13+23].max(axis=0))  # <<== MAX
        # species_score_curr = np.array(df_seltab.iloc[:, 13:21+1].sum(axis=0))  # <<== sum
        
        # keep large scores only > 0.2 # <================= Score threshold
        # species_score_curr = species_score_curr*(species_score_curr>score_thre) 
        species_score_curr = species_score_curr*(species_score_curr>=0.00001) 

        if species_score_curr.size == 0:
            species_score_curr = np.zeros(23)
            species_pred_curr = 0
        elif species_score_curr.sum() == 0.0:
            species_pred_curr = 0
        else:
            species_pred_curr = np.argmax(species_score_curr[:23])
                    
        species_score.append(species_score_curr)  
        species_pred.append(species_pred_curr) 
        seltab_list_tot.append(ss)

species_true = np.concatenate(species_true_tot)
species_score_species = np.vstack(species_score)


From 1705_20170901_0352 to 1705_20170901_0357
mm0: 1136
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035241_084.txt
mm1: 1141
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035741_084.txt
True species id: 5 SPT
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035241_084.txt
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035341_084.txt
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035441_084.txt
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035541_084.txt
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035641_084.txt
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170901_035741_084.txt

From 1705_20170902_0321 to 1705_20170902_0442
mm0: 1762
/home/ys587/__Data/__whistle/__whislte_30_species/__seltab_out/1705_20170902_032126_575.txt
mm1: 1843
/home/ys587/__D

In [18]:
t1=confusion_matrix(species_true, species_pred, labels=list(range(23)))

In [19]:
# confusion matrix
# from sklearn.metrics import confusion_matrix
print(species_list)
print(confusion_matrix(species_true, species_pred, labels=list(range(23)))) # labels=species_list

['NO', 'BD', 'MH', 'CD', 'STR', 'SPT', 'SPIN', 'PLT', 'RD', 'RT', 'WSD', 'FKW', 'BEL', 'KW', 'WBD', 'DUSK', 'FRA', 'PKW', 'LPLT', 'NAR', 'CLY', 'SPE', 'ASP']
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  2  20   0   0   0   0   2  12   0  51   0   1   0   0   0   0   0   0
    0   0   1   2   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [ 54  53   0   6  17   7   1  18   1  36   3   1   0   3   2   0   0   0
    0   0  29   0   0]
 [243  76   1   5   6   6  24   7   1  67   1   5   0   4   1   0   7   0
    0   0  10   2   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  2   0   0   0   0   0   1   3   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [197  29   0   1   2   3  12   5  14  15   1   4   0   5   3   0 

In [ ]:
pd.Series(species_true).value_counts()

In [ ]:
# make a dataframe

In [ ]:
seltab_file = []
for ss in seltab_list_tot:
    seltab_file.append(os.path.basename(ss))

In [ ]:
detect_1705_result = pd.DataFrame({'File': seltab_file, 'Truth': species_true, 'Pred':species_pred, 
                                   'Max': species_score_species.max(axis=1),
                                  'BD': species_score_species[:, 0],
                                   'LCD': species_score_species[:, 1],
                                   'SCD': species_score_species[:, 2],
                                   'STR': species_score_species[:, 3],
                                   'SPT': species_score_species[:, 4],
                                   'SPIN': species_score_species[:, 5],
                                   'PLT': species_score_species[:, 6],
                                   'RT': species_score_species[:, 7],
                                   'FKW': species_score_species[:, 8],
                                   'Path': seltab_list_tot,
                                  }) 
detect_1705_result.to_csv('/home/ys587/__Data/__whistle/__whistle_dclde2020/__seltab_out/1705_results.csv', index=False)

# Deployment 1706

In [ ]:
detection_1706 =  pd.read_excel(seltab_truth, sheet_name='1706_OdontoceteDetections')

In [ ]:
deploy='1706'

species_true_tot = []
species_pred = []
species_score = []
seltab_list_tot = []
# For each acoustic encounter
# for index, row in detection_1706_species_small.iterrows():
for index, row in detection_1706_species.iterrows():
    min_list0 = pd.DataFrame(columns=['NULL'], index=pd.date_range(row['DetectionTimeStart_UTC'], row['DetectionTimeEnd_UTC'],freq='1T')).index.strftime('%Y%m%d_%H%M').tolist()
    min_list = [deploy+'_'+mm for mm in min_list0]
    # ind_start = seltab_list_1706.index(min_list[0])
    print('\nFrom '+min_list[0]+' to '+min_list[-1])
    
    # find start and end indices
    for mm0 in range(len(seltab_list_1706)):
        mm = seltab_list_1706[mm0]
        if min_list[0] in mm:
            print('mm0: '+str(mm0))
            print(mm)
            for mm1 in range(mm0+1, mm0+120):
                mm_end = seltab_list_1706[mm1]
                if  min_list[-1] in mm_end:
                    print('mm1: '+str(mm1))
                    print(mm_end)
                    break
            break
    
    # truth species
    species_true_id = species_dclde2020[row['Species1ID']]
    print('True species id: '+str(species_true_id)+' '+species_list[species_true_id])
    species_true_encounter = np.ones(mm1-mm0+1)*species_true_id
    species_true_tot.append(species_true_encounter)  # <=
    
    
    # extract seltab filenames
    seltab_list = seltab_list_1706[mm0:mm1+1]
    for ss in seltab_list:
        print(ss)
        df_seltab = pd.read_csv(ss, delimiter='\t', header=0)
        species_score_curr = np.array(df_seltab.iloc[:, 13:21+1].max(axis=0))  # <<== MAX
        # species_score_curr = np.array(df_seltab.iloc[:, 13:21+1].sum(axis=0))  # <<== sum
        
        # keep large scores only > 0.2 # <================= Score threshold
        species_score_curr = species_score_curr*(species_score_curr>score_thre) 
        # species_score_curr = species_score_curr*(species_score_curr>=0.2) 

        if species_score_curr.size == 0:
            species_score_curr = np.zeros(9)
            species_pred_curr = 9
        elif species_score_curr.sum() == 0.0:
            species_pred_curr = 9
        else:
            species_pred_curr = np.argmax(species_score_curr[:9])
                    
        species_score.append(species_score_curr)  
        species_pred.append(species_pred_curr) 
        seltab_list_tot.append(ss)

species_true = np.concatenate(species_true_tot)
species_score_species = np.vstack(species_score)

In [ ]:
# confusion matrix
print(species_list)
print(confusion_matrix(species_true, species_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])) # labels=species_list

In [ ]:
pd.Series(species_true).value_counts()

In [ ]:
# make a dataframe

In [ ]:
seltab_file = []
for ss in seltab_list_tot:
    seltab_file.append(os.path.basename(ss))

In [ ]:
detect_1706_result = pd.DataFrame({'File': seltab_file, 'Truth': species_true, 'Pred':species_pred, 
                                   'Max': species_score_species.max(axis=1),
                                  'BD': species_score_species[:, 0],
                                   'LCD': species_score_species[:, 1],
                                   'SCD': species_score_species[:, 2],
                                   'STR': species_score_species[:, 3],
                                   'SPT': species_score_species[:, 4],
                                   'SPIN': species_score_species[:, 5],
                                   'PLT': species_score_species[:, 6],
                                   'RT': species_score_species[:, 7],
                                   'FKW': species_score_species[:, 8],
                                   'Path': seltab_list_tot,
                                  }) 
detect_1706_result.to_csv('/home/ys587/__Data/__whistle/__whistle_dclde2020/__seltab_out/1706_results.csv', index=False)